## Data Mining and Machine Learning
## Edgar Acuna
## Imputation III
### mean/median/mode and k-nn imputation
### Datasets: Titanic, Breastw y Segment 

In [172]:
import pandas as pd
import numpy as np
from sklearn.preprocessing  import LabelEncoder 

In [173]:
#Ejemplo1. Leyendo los datos de Titanic
titanic=pd.read_csv('http://academic.uprm.edu/eacuna/titanic.csv',header=0,sep=',',na_values='')
titanic.info()
titanic.head(39)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


PassengerId  Pclass                                               Name  \
0             1       3                            Braund, Mr. Owen Harris   
1             2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2             3       3                             Heikkinen, Miss. Laina   
3             4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4             5       3                           Allen, Mr. William Henry   
5             6       3                                   Moran, Mr. James   
6             7       1                            McCarthy, Mr. Timothy J   
7             8       3                     Palsson, Master. Gosta Leonard   
8             9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9            10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10           11       3                    Sandstrom, Miss. Marguerite Rut   
11           12       1                           Bonnell, Miss. Elizabeth   
12           13       3                     Saundercock, Mr. William Henry   
13           14       3                        Andersson, Mr. Anders Johan   
14           15       3               Vestrom, Miss. Hulda Amanda Adolfina   
15           16       2                   Hewlett, Mrs. (Mary D Kingcome)    
16           17       3                               Rice, Master. Eugene   
17           18       2                       Williams, Mr. Charles Eugene   
18           19       3  Vander Planke, Mrs. Julius (Emelia Maria Vande...   
19           20       3                            Masselmani, Mrs. Fatima   
20           21       2                               Fynney, Mr. Joseph J   
21           22       2                              Beesley, Mr. Lawrence   
22           23       3                        McGowan, Miss. Anna "Annie"   
23           24       1                       Sloper, Mr. William Thompson   
24           25       3                      Palsson, Miss. Torborg Danira   
25           26       3  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
26           27       3                            Emir, Mr. Farred Chehab   
27           28       1                     Fortune, Mr. Charles Alexander   
28           29       3                      O'Dwyer, Miss. Ellen "Nellie"   
29           30       3                                Todoroff, Mr. Lalio   
30           31       1                           Uruchurtu, Don. Manuel E   
31           32       1     Spencer, Mrs. William Augustus (Marie Eugenie)   
32           33       3                           Glynn, Miss. Mary Agatha   
33           34       2                              Wheadon, Mr. Edward H   
34           35       1                            Meyer, Mr. Edgar Joseph   
35           36       1                     Holverson, Mr. Alexander Oskar   
36           37       3                                   Mamee, Mr. Hanna   
37           38       3                           Cann, Mr. Ernest Charles   
38           39       3                 Vander Planke, Miss. Augusta Maria   

       Sex   Age  SibSp  Parch            Ticket      Fare        Cabin  \
0     male  22.0      1      0         A/5 21171    7.2500          NaN   
1   female  38.0      1      0          PC 17599   71.2833          C85   
2   female  26.0      0      0  STON/O2. 3101282    7.9250          NaN   
3   female  35.0      1      0            113803   53.1000         C123   
4     male  35.0      0      0            373450    8.0500          NaN   
5     male   NaN      0      0            330877    8.4583          NaN   
6     male  54.0      0      0             17463   51.8625          E46   
7     male   2.0      3      1            349909   21.0750          NaN   
8   female  27.0      0      2            347742   11.1333          NaN   
9   female  14.0      1      0            237736   30.0708          NaN   
10  female   4.0      1      1           PP 9549   16.7000

Deleting columns: PassengerID, Name and cabin(it has a lot of missing values) 

In [174]:
titanic1=titanic.iloc[:,[1,3,4,5,6,8,10]]
titanic1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1046 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Fare        1308 non-null float64
Embarked    1307 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 71.7+ KB


Hay missings en las variables Age, Fare que son continuas y en Embarked que es categoria

In [175]:
#computing the values to  be used in the imputation
t3=titanic1['Age'].mean()
t6=titanic1['Fare'].mean()
t7=titanic1['Embarked'].mode().iloc[0]

In [176]:
#Performing the imputacion
values = {'Age': t3, 'Fare': t6, 'Embarked': t7}
titanic2=titanic1.fillna(value=values)
titanic2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1309 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Fare        1309 non-null float64
Embarked    1309 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 71.7+ KB


Imputando los missing values por la media si el atributo es continuo y por la moda si el atributo es nominal

In [177]:
#Imputando los missing values por la media si el atributo es continuo
#y por la moda si el atributo es nominal
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

In [178]:
xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1309 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Fare        1309 non-null float64
Embarked    1309 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 71.7+ KB


In [179]:
#Imputando los missing values por la mediana si el atributo es continuo
#y por la moda si el atributo es nominal

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

### IMPUTACION k-nn

In [180]:
#imputando por knn
from sklearn import neighbors

class ImputeLearn(object):
    def __init__(self, mat):
        self.mat = np.array(mat)
        self.mat_isnan = np.isnan(self.mat)
        self.w = np.where(np.isnan(self.mat))

    def impute(self, learner):
        ximp = self.mat.copy()
        for i in np.arange(0, len(self.w[0])):
            n = self.w[0][i] # row where the nan is
            p = self.w[1][i] # column where the nan is
            col_isnan = self.mat_isnan[n, :] # empty columns in row n
            train = np.delete(self.mat, n, axis = 0) # remove row n to obtain a training set
            train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
            target = train_nonan[:, p] # vector to be predicted
            feature = train_nonan[:, ~col_isnan] # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(self.mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
        
        return(ximp)

#### Ejemplo 2. k-nn imputation applied to the  Breast-wisconsin dataset

In [181]:
#Reading the data from the UCI
breastdf=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",header=None, sep=",",na_values=['?'])
#breastdf=pd.read_csv("c://PW-PR/breast-cancer-wisconsin.data",header=None, sep=",",na_values=['?'])

In [182]:
breastdf.columns=['idx','v1','v2','v3','v4','v5','v6','v7','v8','v9','class']

In [183]:
breastdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
idx      699 non-null int64
v1       699 non-null int64
v2       699 non-null int64
v3       699 non-null int64
v4       699 non-null int64
v5       699 non-null int64
v6       683 non-null float64
v7       699 non-null int64
v8       699 non-null int64
v9       699 non-null int64
class    699 non-null int64
dtypes: float64(1), int64(10)
memory usage: 60.1 KB


In [184]:
breastdf.head()

idx  v1  v2  v3  v4  v5    v6  v7  v8  v9  class
0  1000025   5   1   1   1   2   1.0   3   1   1      2
1  1002945   5   4   4   5   7  10.0   3   2   1      2
2  1015425   3   1   1   1   2   2.0   3   1   1      2
3  1016277   6   8   8   1   3   4.0   3   7   1      2
4  1017023   4   1   1   3   2   1.0   3   1   1      2

In [185]:
breast1=breastdf.iloc[:,1:11]

In [186]:
breast1.head()

v1  v2  v3  v4  v5    v6  v7  v8  v9  class
0   5   1   1   1   2   1.0   3   1   1      2
1   5   4   4   5   7  10.0   3   2   1      2
2   3   1   1   1   2   2.0   3   1   1      2
3   6   8   8   1   3   4.0   3   7   1      2
4   4   1   1   3   2   1.0   3   1   1      2

In [187]:
b1=np.array(breast1)

In [188]:
data_imp = ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [189]:
completo5=pd.DataFrame(data_imp)

In [190]:
completo5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
0    699 non-null float64
1    699 non-null float64
2    699 non-null float64
3    699 non-null float64
4    699 non-null float64
5    699 non-null float64
6    699 non-null float64
7    699 non-null float64
8    699 non-null float64
9    699 non-null float64
dtypes: float64(10)
memory usage: 54.7 KB


#### Ejemplo. Imputacion k-nn a Titanic. Hay que convertir las variables categoticas a numericas porque knn-imputacion solo esta implementada para variables numericas 

In [191]:
#Convirtiendo las variables nominales en numericas, debido a que 
#la imputacion knn de python no trabaja con variables mezcladas
#Debe hacerse con cuidados porque el atributo categorico continen missing vakue y el LabelEncoder 
#puede asignarle un valor en lygar  de ignorarlo
titanic3=titanic1
mask = titanic1.isnull()
titanic1=titanic1.astype(str).apply(LabelEncoder().fit_transform)
titanic1.where(~mask, titanic3)

Pclass  Sex  Age  SibSp  Parch Fare Embarked
0          2    1   25      1      0  195        2
1          0    0   51      1      0  227        0
2          2    0   32      0      0  225        2
3          0    0   47      1      0  160        2
4          2    1   47      0      0  240        2
5          2    1  NaN      0      0  248        1
6          0    1   72      0      0  156        2
7          2    1   21      3      1   75        2
8          2    0   34      0      2    8        2
9          1    0   13      1      0  117        0
10         2    0   54      1      1   60        2
11         0    0   77      0      0  101        2
12         2    1   22      0      0  240        2
13         2    1   53      1      5  121        2
14         2    0   13      0      0  220        2
15         1    0   73      0      0   58        2
16         2    1   21      4      1  113        1
17         1    1  NaN      0      0   24        2
18         2    0   41      1      0   64        2
19         2    0  NaN      0      0  193        0
20         1    1   47      0      0   96        2
21         1    1   45      0      0   24        2
22         2    0   15      0      0  239        1
23         0    1   35      0      0  134        2
24         2    0   95      3      1   75        2
25         2    0   51      1      5  122        2
26         2    1  NaN      0      0  193        0
27         0    1   20      3      2  103        2
28         2    0  NaN      0      0  222        1
29         2    1  NaN      0      0  224        2
...      ...  ...  ...    ...    ...  ...      ...
1279       2    1   24      0      0  211        1
1280       2    1   79      3      1   75        2
1281       0    1   27      0      0  280        2
1282       0    0   69      0      1  139        2
1283       2    1   12      0      2   71        2
1284       1    1   64      0      0    3        2
1285       2    1   37      3      1   79        2
1286       0    0   18      1      0  178        2
1287       2    1   29      0      0  195        1
1288       0    0   65      1      1  237        0
1289       2    1   25      0      0  212        2
1290       2    1   41      0      0  208        1
1291       0    0   39      0      0   61        2
1292       1    1   51      1      0   74        2
1293       0    0   25      0      1  169        0
1294       0    1   17      0      0  149        2
1295       0    1   59      1      0  106        0
1296       1    1   22      0      0   30        0
1297       1    1   27      1      0    3        2
1298       0    1   68      1      1   78        0
1299       2    0  NaN      0      0  205        1
1300       2    0   38      1      1   27        2
1301       2    0  NaN      0      0  211        1
1302       0    0   50      1      0  278        1
1303       2    0   35      0      0  212        2
1304       2    1  NaN      0      0  240        2
1305       0    0   53      0      0    7        0
1306       2    1   52      0      0  195        2
1307       2    1  NaN      0      0  240        2
1308       2    1  NaN      1      1   80        0

[1309 rows x 7 columns]

In [192]:
b1=np.array(titanic1)

In [193]:
titanic_knnimp=ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [194]:
completo=pd.DataFrame(titanic_knnimp)
completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
0    1309 non-null int32
1    1309 non-null int32
2    1309 non-null int32
3    1309 non-null int32
4    1309 non-null int32
5    1309 non-null int32
6    1309 non-null int32
dtypes: int32(7)
memory usage: 35.9 KB


#### Example 2. Simulating misising values and  imputing them. Dataset: segment from the UCI

This dataset is an image segmentation database similar to a database already present in the repository 
(Image segmentation database) but in a slightly different form. The instances were drawn randomly from a 
database of 7 outdoor images. 
The images were handsegmented to create a classification for every pixel. Each instance is a 3x3 region. 


In [195]:
#seg=pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/segment/segment.dat',header=None, delim_whitespace=True)
seg=seg=pd.read_table('c://PW-PR/segment.dat',header=None, delim_whitespace=True)
seg.columns=['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','class']

In [196]:
seg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2310 non-null float64
v2       2310 non-null float64
v3       2310 non-null int64
v4       2310 non-null float64
v5       2310 non-null float64
v6       2310 non-null float64
v7       2310 non-null float64
v8       2310 non-null float64
v9       2310 non-null float64
v10      2310 non-null float64
v11      2310 non-null float64
v12      2310 non-null float64
v13      2310 non-null float64
v14      2310 non-null float64
v15      2310 non-null float64
v16      2310 non-null float64
v17      2310 non-null float64
v18      2310 non-null float64
v19      2310 non-null float64
class    2310 non-null int64
dtypes: float64(18), int64(2)
memory usage: 361.0 KB


Todos las variables de segment son cuantitativas.

In [197]:
#simulando un 5% de celdas missing
import random
seg5=seg.copy()
seg5= seg5.stack().sample(frac=0.95).unstack().reindex(index=seg5.index, columns=seg5.columns)


In [198]:
seg5.head()

v1     v2   v3        v4   v5        v6        v7        v8        v9  \
0  218.0  178.0  9.0  0.111111  0.0  0.833333  0.547722  1.111109  0.544331   
1  113.0  130.0  9.0  0.000000  0.0  0.277778  0.250924  0.333333  0.365148   
2  202.0   41.0  9.0  0.000000  0.0  0.944448  0.772202  1.111112       NaN   
3   32.0  173.0  9.0  0.000000  0.0  1.722222  1.781593  9.000000  6.749488   
4   61.0  197.0  9.0  0.000000  0.0  1.444444  1.515353  2.611111  1.925463   

          v10         v11         v12         v13        v14       v15  \
0   59.629630   52.444443   75.222220   51.222220 -21.555555  46.77778   
1    0.888889    0.000000    2.555556    0.111111  -2.666667   5.00000   
2  123.037040  111.888885  139.777790  117.444440        NaN  50.22222   
3   43.592594         NaN   52.888890   38.333336 -12.111111  27.88889   
4   49.592594   44.222220   61.555557   43.000000 -16.111110  35.88889   

         v16         v17       v18       v19  class  
0 -25.222221   75.222220       NaN -2.040554    6.0  
1  -2.333333         NaN  1.000000 -2.123254    3.0  
2        NaN  139.777790  0.199347 -2.299918    2.0  
3 -15.777778   52.888890  0.266914 -1.998858    6.0  
4 -19.777779   61.555557  0.302925 -2.022274    6.0

In [199]:
seg5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2189 non-null float64
v2       2198 non-null float64
v3       2187 non-null float64
v4       2195 non-null float64
v5       2186 non-null float64
v6       2198 non-null float64
v7       2202 non-null float64
v8       2219 non-null float64
v9       2187 non-null float64
v10      2199 non-null float64
v11      2225 non-null float64
v12      2184 non-null float64
v13      2213 non-null float64
v14      2195 non-null float64
v15      2193 non-null float64
v16      2184 non-null float64
v17      2189 non-null float64
v18      2180 non-null float64
v19      2180 non-null float64
class    2187 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [200]:
#imputando por la media
seg5.mean=seg5.apply(lambda x: x.fillna(x.mean()),axis=0)

In [201]:
#Imputando por la mediana
seg5.median=seg5.apply(lambda x: x.fillna(x.median()),axis=0)

In [202]:
b1=np.array(seg5)

In [203]:
data_imp = ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [204]:
completoknn5=pd.DataFrame(data_imp)

In [205]:
completoknn5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
0     2310 non-null float64
1     2310 non-null float64
2     2310 non-null float64
3     2310 non-null float64
4     2310 non-null float64
5     2310 non-null float64
6     2310 non-null float64
7     2310 non-null float64
8     2310 non-null float64
9     2310 non-null float64
10    2310 non-null float64
11    2310 non-null float64
12    2310 non-null float64
13    2310 non-null float64
14    2310 non-null float64
15    2310 non-null float64
16    2310 non-null float64
17    2310 non-null float64
18    2310 non-null float64
19    2310 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [206]:
from sklearn.metrics import mean_squared_error
#Calculando el cuadrado medio del error para imputacion por la media
completo5mean=pd.DataFrame(seg5.mean)
mean_squared_error(seg,completo5mean)

44.92075474300479

In [207]:
#Calculando el cuadrado medio del error para imputacion por la mediana
completo5median=pd.DataFrame(seg5.median)
mean_squared_error(seg,completo5median)

47.957936877205505

In [208]:
#Calculando el cuadrado medio del error para imputacion knn
mean_squared_error(seg,completoknn5)

16.940110452664737

Notice that the best imputation method was k-nn

In [209]:
#simulando un 20% de celdas missing
seg20=seg.copy()
seg20= seg20.stack().sample(frac=0.80).unstack().reindex(index=seg20.index, columns=seg20.columns)
seg20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       1839 non-null float64
v2       1822 non-null float64
v3       1852 non-null float64
v4       1878 non-null float64
v5       1867 non-null float64
v6       1837 non-null float64
v7       1857 non-null float64
v8       1859 non-null float64
v9       1849 non-null float64
v10      1854 non-null float64
v11      1847 non-null float64
v12      1831 non-null float64
v13      1816 non-null float64
v14      1868 non-null float64
v15      1837 non-null float64
v16      1858 non-null float64
v17      1848 non-null float64
v18      1865 non-null float64
v19      1831 non-null float64
class    1845 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [210]:
#hallando el numero de filas que tienen missings
sum(seg20.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

2287

In [211]:
#imputando por la media al 20%
seg20.mean=seg20.apply(lambda x: x.fillna(x.mean()),axis=0)
completo20mean=pd.DataFrame(seg20.mean)
mean_squared_error(seg,completo20mean)

284.2609262599753

In [212]:
#imputando por la mediana al 20%
seg20.median=seg20.apply(lambda x: x.fillna(x.median()),axis=0)
completo20median=pd.DataFrame(seg20.median)
mean_squared_error(seg,completo20median)

300.0634769279861

In [213]:
b2=np.array(seg20)

In [214]:
data_imp = ImputeLearn(b2).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [215]:
completoknn20=pd.DataFrame(data_imp)
mean_squared_error(seg,completoknn20)

241.24915578860933

Once again the best imputation method was k-nn